In [ ]:
import boto3
import threading
import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from botocore.exceptions import ClientError

# Initialize the Timestream client
client = boto3.client('timestream-query', region_name='us-east-2')

# List of different Timestream queries
queries = [
    f'select memory from "devops"."sample_devops" where time > ago(10m) and hostname=\'host1\' order by time desc limit 1',
]

# Function to run a single query and measure its duration
def run_query(query, worker_id, worker_results,throttling_results):
    start_time = time.time()
    worker_results[worker_id] = []
    end_time = time.time() + duration_seconds
    throttling_results[worker_id] = []
    throttling_count = []
    results_dict = {}
    while time.time() < end_time:
        try:
            start = time.time()
            response = client.query(QueryString=query)
            duration = time.time() - start
            worker_results[worker_id].append((duration))
        except ClientError as e:
            if e.response['Error']['Code'] == 'ThrottlingException':
                throttling_results[worker_id].append("ThrottlingException")
            else:
                raise e 


# Function to run queries in parallel using threads
def run_parallel_queries(duration_seconds, queries, users):
    end_time = time.time() + duration_seconds
    total_queries = 0
    query_durations = []
    throttling_count = {}
    worker_results = {}
    throttling_results = {}
    threads = []

    print(f"\nLoad Test Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


    for i in range(users):
        worker_id = i
        thread = threading.Thread(target=run_query, args=(queries[i % len(queries)], worker_id, worker_results, throttling_results))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    
    query_durations = [value for worker_results in worker_results.values() for value in worker_results]
    total_queries = len(query_durations)
    throttling_count=(sum(len(exceptions_list) for exceptions_list in throttling_results.values()))
    

    print(f"Load Test End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    if query_durations:
        p50 = np.percentile(query_durations, 50)
        p90 = np.percentile(query_durations, 90)
        p99 = np.percentile(query_durations, 99)
    else:
        p50 = p90 = p99 = None 

    
    return total_queries, p50, p90, p99, throttling_count


users_count = [1,2,3,4,5,6,7,9,11,13,15,17,21]  
duration_seconds = 60

results = []

# Run the tests for different worker counts
for users in users_count:
    total_queries, p50, p90, p99, throttling_count = run_parallel_queries(duration_seconds, queries, users)
    results.append((users, total_queries, p50, p90, p99, throttling_count))
    print(f"num_users: {users}")
    print(f"Total number of queries run in {duration_seconds} seconds: {total_queries}")
    print(f"p50 (50th percentile) of query durations: {p50:.2f} seconds")
    print(f"p90 (90th percentile) of query durations: {p90:.2f} seconds")
    print(f"p99 (99th percentile) of query durations: {p99:.2f} seconds")
    print(f"Throttling count: {throttling_count}")



In [ ]:
total_queries = [result[1] for result in results]
p50s = [result[2] for result in results]
p90s = [result[3] for result in results]
p99s = [result[4] for result in results]
throttling_counts = [result[5] for result in results]

plt.figure(figsize=(12, 8))

# Plot latency percentiles
plt.subplot(3, 1, 1)
plt.plot(users_count, p50s, label='p50')
plt.plot(users_count, p90s, label='p90')
plt.plot(users_count, p99s, label='p99')
plt.xlabel('Number of Users')
plt.ylabel('Latency (seconds)')
plt.xticks(users_count)
plt.title('Latency Percentiles')
plt.legend()

# Plot Queries Per Minute (QPM)
plt.subplot(3, 1, 2)
qpm = [q / (duration_seconds / 60) for q in total_queries]
plt.plot(users_count, qpm, label='Queries Per Minute (QPM)')
plt.xlabel('Number of Users')
plt.ylabel('Queries Per Minute')
plt.xticks(users_count)
plt.title('Queries Per Minute')
plt.legend()

# Plot Throttling Counts
plt.subplot(3, 1, 3)
plt.plot(users_count, throttling_counts, label='Throttling Count', color='red')
plt.xlabel('Number of Users')
plt.ylabel('Throttling Count')
plt.xticks(users_count)
plt.title('Throttling Count')
plt.legend()

plt.tight_layout()
plt.show() 